In [1]:
# fisher girl
from PIL import Image, ImageFilter, ImageEnhance

# sharpening
img = Image.open('zhuhai_fisher_girl.jpg')
img_output = img.filter(ImageFilter.SHARPEN)

# contrast adjust
enhancer = ImageEnhance.Contrast(img_output)
contrasted_image = enhancer.enhance(1.25)

contrasted_image.save('modified_girl.jpg')
contrasted_image.show()

In [22]:
# crowd
import cv2
import numpy as np

# sobel filter
image = cv2.imread('crowd_looking.png', cv2.IMREAD_GRAYSCALE)

sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)  # x direction
sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)  # y direction

sobel_combined = np.sqrt(sobel_x**2 + sobel_y**2)
sobel_combined = np.uint8(sobel_combined)

# contrast and brightness adjust
sobel_combined = cv2.convertScaleAbs(sobel_combined, alpha=2.0, beta=0)

# image flip
sobel_combined = cv2.flip(sobel_combined, 1)

# image resize
result = cv2.resize(sobel_combined, (440, 362), interpolation=cv2.INTER_AREA)

cv2.imwrite('modified_crowd.png', result)

True

In [15]:
# mermaid
from PIL import Image, ImageEnhance
import numpy as np

img = Image.open('mermaid.png')

# saturation adjustment
img_saturation = ImageEnhance.Color(img)
result = img_saturation.enhance(2.0)

result.save('modified_mermaid.png')

In [53]:
# portal
from PIL import Image, ImageSequence

# resizing the gif
filename = "portal.gif"
new_width = 200
new_height = 200

with Image.open(filename) as im:
    frames = []
    for frame in ImageSequence.Iterator(im):
        frame = frame.resize((new_width, new_height))
        frames.append(frame)
    print(f"length of freames {len(frames)}")
    frames[0].save('modified_portal.gif', save_all=True, append_images=frames[1:])

length of freames 30


In [14]:
# floating island animation
from PIL import Image
import numpy as np

image = Image.open('floating_island.png')

frames = 24  
amplitude = 10

# Create frames
gif_frames = []
for i in range(frames):
    offset = int(amplitude * np.sin(2 * np.pi * i / frames))
    frame = Image.new('RGBA', image.size, (255, 255, 255, 0))
    frame.paste(image, (0, offset), image)
    gif_frames.append(frame)

# Save the frames
gif_frames[0].save(
    'floating_island.gif',
    save_all=True,
    append_images=gif_frames[1:],
    duration=100,
    loop=0,
    disposal=2
)

In [59]:
# overlay
from PIL import Image

img = Image.open('modified_girl.jpg')
img2 = Image.open('modified_crowd.png')
img3 = Image.open('modified_mermaid.png')

img.paste(img2, (0, 570), mask = img2)
img.paste(img3, (400, 650), mask = img3)

img.save('result_v2.jpg')

In [58]:
from PIL import Image, ImageDraw
from moviepy.editor import ImageSequenceClip, AudioFileClip
import numpy as np

# vignette effect to simulate difficulty staying conscious
def add_vignette(image, strength=0.75):
    width, height = image.size
    mask = Image.new("L", (width, height))
    draw = ImageDraw.Draw(mask)
    
    for y in range(height):
        for x in range(width):
            dx = (x - width / 2.0) ** 2
            dy = (y - height / 2.0) ** 2
            distance = min(1.0, (dx + dy) / ((width / 2.0) ** 2 + (height / 2.0) ** 2))
            darkness = int(255 * (1 - distance * strength))
            draw.point((x, y), fill=darkness)
    
    black_image = Image.new("RGB", (width, height), "black")
    vignette_image = Image.composite(image, black_image, mask)
    
    return vignette_image

base_image = Image.open('result_v2.jpg').convert("RGBA")
island_gif = Image.open('floating_island.gif')
portal_gif = Image.open('modified_portal.gif')

portal_frames = []
try:
    while True:
        portal_frames.append(portal_gif.copy().convert("RGBA"))
        portal_gif.seek(portal_gif.tell() + 1)
except EOFError:
    pass

frames = 24  
amplitude = 10  
duration = 3 
vignette_frequency = 3

# creating frames for GIF
overlay_frames = []
for i in range(frames):
    offset = int(amplitude * np.sin(2 * np.pi * i / frames))
    frame = base_image.copy()

    temp_island_frame = island_gif.copy().convert("RGBA")
    frame.paste(temp_island_frame, (700, offset), temp_island_frame)

    temp_portal_frame = portal_frames[i % len(portal_frames)]
    frame.paste(temp_portal_frame, (400, 130), temp_portal_frame)
    
    vignette_strength = 0.75 * (1 + np.sin(2 * np.pi * vignette_frequency * i / frames)) / 2
    frame_with_vignette = add_vignette(frame, strength=vignette_strength)
    
    overlay_frames.append(frame_with_vignette)

clip = ImageSequenceClip([np.array(frame) for frame in overlay_frames], fps=frames/duration)

audio = AudioFileClip('out_of_breath.mp3').set_duration(clip.duration)
clip = clip.set_audio(audio)

clip.write_videofile('result_v2.mp4', codec='libx264', audio_codec='aac')

Moviepy - Building video result_v2.mp4.
MoviePy - Writing audio in result_v2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video result_v2.mp4



Moviepy - Done !
Moviepy - video ready result_v2.mp4
